In [2]:
import os
import csv
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver import Edge
from selenium.webdriver.common.by import By
import re
import datetime
import time

In [1]:
import os
import csv
import time
import datetime
import json
from selenium import webdriver
from bs4 import BeautifulSoup

# Original URL template
original_url = "https://www.wunderground.com/history/daily/us/nc/charlotte/HOU/date/2016-2-28"

# Start and end dates
start_date = datetime.datetime(2020, 1, 1)
end_date = datetime.datetime(2023, 12, 30)

# List of codes to replace HOU with
codes = ["ATL", "CLT", "DEN", "DFW", "EWR", "IAH", "JFK", "LAS", "LAX", "MCO", "MIA", "ORD", "PHX", "SEA", "SFO"]

# Initialize WebDriver outside the loop
options = webdriver.EdgeOptions()
options.add_argument('--headless')
driver = webdriver.Edge()

# Checkpoint file path
checkpoint_file = 'checkpoint.json'

# Load the last checkpoint if it exists
last_checkpoint = {}
if os.path.exists(checkpoint_file):
    with open(checkpoint_file, 'r') as f:
        last_checkpoint = json.load(f)
        if not isinstance(last_checkpoint, dict):
            last_checkpoint = {}

# Get the last scraped area code and date
last_code = last_checkpoint.get('last_code', '')
last_date = datetime.datetime.strptime(last_checkpoint.get('last_date', '2020-01-01'), '%Y-%m-%d')

# Iterate over each code in the list
for code in codes:
    # Create a folder for the current year if it doesn't exist
    year_folder = str(start_date.year)
    if not os.path.exists(year_folder):
        os.makedirs(year_folder)
    
    # Iterate over dates until end date
    current_date = start_date
    while current_date <= end_date:
        # If the code has already been scraped, skip it
        if last_code and code < last_code:
            current_date = end_date + datetime.timedelta(days=1)  # Skip to end_date to start from the next code
            break
        
        # If the date has already been scraped, skip it
        if current_date <= last_date and code == last_code:
            current_date += datetime.timedelta(days=1)
            continue
        
        # Format date as yyyy-mm-dd
        formatted_date = current_date.strftime("%Y-%m-%d")
        
        # Replace HOU with the current code and formatted date in the URL
        modified_url = original_url.replace("HOU", code).replace("2016-2-28", formatted_date)
        
        # Visit the modified URL
        driver.get(modified_url)
        time.sleep(4)
        
        # Get the HTML content of the page after loading all flights
        html = driver.page_source
        
        # Parse the HTML content
        soup = BeautifulSoup(html, 'html.parser')
        
        # Find the table body
        tbody = soup.find('tbody', role='rowgroup')
        
        # Check if table body exists
        if tbody:
            # Find all rows in the table
            rows = tbody.find_all('tr')
            
            # Define CSV file name based on year and month
            csv_file_name = os.path.join(year_folder, f"{current_date.strftime('%Y-%m')}.csv")
            
            # Open the CSV file in append mode
            with open(csv_file_name, mode='a', newline='', encoding='utf-8') as file:
                writer = csv.writer(file)
                
                # Write each row to the CSV file
                for row in rows:
                    # Find all cells in the row
                    cells = row.find_all('td')
                    
                    # Extract data from cells
                    row_data = [code, formatted_date]  # Append code and formatted date
                    row_data.extend([cell.text.strip() for cell in cells])  # Append cell data
                    writer.writerow(row_data)
        else:
            print("Table body not found.")
        
        # Update the last checkpoint if code has changed
        if code != last_code:
            last_checkpoint['last_code'] = code
            last_checkpoint['last_date'] = current_date.strftime('%Y-%m-%d')
            with open(checkpoint_file, 'w') as f:
                json.dump(last_checkpoint, f)
        
        current_date += datetime.timedelta(days=1)
        time.sleep(2)

# Close the WebDriver after the loop completes
driver.quit()


Table body not found.


KeyboardInterrupt: 